In [1]:
import xymass
import numpy as np
import scipy
import astropy.units as u
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
n_object=100000 #sample size
f_binary=0.5 #binary fraction

In [3]:
#draw sample of psoitions from specified density profile.
#Since these are positions of center of masses in binary systems, 
#r_scale must be specified with units via astropy

r2d=xymass.sample_r2d(size=n_object,model='plum',r_scale=100.*u.pc) 

/Users/walker/opt/anaconda3/lib/python3.9/site-packages/xymass.py:50: UserWarning: ellipticity and position_angle not specified, assuming ellipticity=0
  warnings.warn('ellipticity and position_angle not specified, assuming ellipticity=0')


In [4]:
#draw sample from specified mass function.
#Since these are masses of primaries in binary systems, must specify units via astropy

m_min=0.1
mass_primary=xymass.sample_imf(size=n_object,model='kroupa',m_min=m_min).mass*u.M_sun

In [5]:
#add binary companions according to specified binary fraction with Raghavan etal. 2010 parameters

r2d_with_binaries_raghavan=xymass.add_binaries(r2d.r_xyz,mass_primary,f_binary=f_binary,\
                                               binary_model='Raghavan2010')

In [ ]:
#add binary companions according to specified binary fraction with Duquennoy & Mayor(1991) parameters

r2d_with_binaries_dm91=xymass.add_binaries(r2d.r_xyz,mass_primary,f_binary=f_binary,binary_model='DM91')

In [ ]:
#add binary companions according to mass ratio, period and eccentricity distributions sampled by the user

#get array of m_secondary / m_primary, sampled from uniform distribution subject to constraint M_2 > 0.1 Msun

q=np.random.uniform(size=n_object,low=m_min/mass_primary.value,high=1.) 

#get array of orbital period (years), sampled from truncated log-normal distribution

period=10.**xymass.sample_normal_truncated(size=n_object,loc=5.03,scale=2.28,\
                                           min_value=-np.inf,max_value=np.inf)/364.25*u.yr 

#get array of orbital eccentricity, sampled from truncated log-normal distribution

eccentricity=xymass.sample_normal_truncated(size=n_object,loc=0.31,scale=0.17,min_value=0.,max_value=1.)
    
r2d_with_binaries_user=xymass.add_binaries(r2d.r_xyz,mass_primary,f_binary=f_binary,q=q,\
                                           period=period,eccentricity=eccentricity) 

In [ ]:
#plot nearest neighbor separation function for each case

fig=plt.figure(figsize=(12,8))
ax1=fig.add_subplot(231)
ax2=fig.add_subplot(232)
ax3=fig.add_subplot(233)
#ax4=fig.add_subplot(224)
fig.subplots_adjust(wspace=0.8,hspace=0.5)

ax=[ax1,ax2,ax3]
title=['Raghavan+2010','D+M 1991','user defined']

i=0
for sample in [r2d_with_binaries_raghavan,r2d_with_binaries_dm91,r2d_with_binaries_user]:
    tree=scipy.spatial.KDTree(sample.r_xyz.value)
    tree_singles=scipy.spatial.KDTree(sample.r_xyz[sample.item==0].value)
    tree_binaries=scipy.spatial.KDTree(sample.r_xyz[sample.item>0].value)
    nn=tree.query(sample.r_xyz.value,k=2)[0].T[1]#gives array of NN distance for each item in sample
    nn_singles=tree_singles.query(sample.r_xyz[sample.item==0].value,k=2)[0].T[1]
    nn_binaries=tree_binaries.query(sample.r_xyz[sample.item>0].value,k=2)[0].T[1]
            
    ax[i].hist(np.log10(nn/r2d.rhalf_2d.value),bins=50,histtype='step',density=False,color='k',label='all')
    ax[i].hist(np.log10(nn_singles/r2d.rhalf_2d.value),bins=50,histtype='step',density=False,color='b',label='singles')
    ax[i].hist(np.log10(nn_binaries/r2d.rhalf_2d.value),bins=50,histtype='step',density=False,color='r',label='binaries')
    ax[i].set_xlabel(r'$\log_{10}[\mathrm{NN sep}\,/\,R_{\rm half}]$',fontsize=10)
    ax[i].set_title(title[i],fontsize=10)
    ax[i].set_ylabel('N')
    ax[i].legend(loc=2,fontsize=8)

    i+=1
    
plt.show()
